## 剩余pytest的操作基本与配置、hook等个性化设置有关，而命令行操作语法与shell语法基本相似，可以自行查阅官方文档，等到具体项目的时候再学，今天先复习一下Selenium

In [6]:
from selenium import webdriver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("http://selenium.dev")
driver.quit()

通过运行上述代码检查环境配置是否有问题，如有问题可以自行查阅文档解决

使用一个官方示例来测试,见using_selenium_tests.py直接运行即可

In [6]:
!conda run -n myPyTest pytest -sv G:\Git_repo\PytestParactice\Day2\using_selenium_tests.py 

============================= test session starts =============================

platform win32 -- Python 3.9.21, pytest-8.3.4, pluggy-1.5.0 -- E:\Anaconda\envs\myPyTest\python.exe

cachedir: .pytest_cache

rootdir: G:\Git_repo\PytestParactice

configfile: pytest.ini

plugins: anyio-4.7.0

collecting ... collected 1 item



using_selenium_tests.py::test_eight_components PASSED



============================== 1 passed in 9.22s ==============================


嘿，给我气笑了，笔者才发现之前pytest一直用的默认conda环境，即base环境在跑测试，不是这个包一直没导入，笔者还发现不了，改成上述命令就好了，！是用来告诉Jupyter，这是命令行语句的

test_options.py 是selenium官方给出的option说明用例，现在来体验一下

In [8]:
!conda run -n myPyTest pytest -sv G:\Git_repo\PytestParactice\Day2\test_options.py::test_page_load_strategy_normal

============================= test session starts =============================

platform win32 -- Python 3.9.21, pytest-8.3.4, pluggy-1.5.0 -- E:\Anaconda\envs\myPyTest\python.exe

cachedir: .pytest_cache

rootdir: G:\Git_repo\PytestParactice

configfile: pytest.ini

plugins: anyio-4.7.0

collecting ... collected 1 item



test_options.py::test_page_load_strategy_normal PASSED



============================== 1 passed in 9.79s ==============================


其中options.add_argument('--no-sandbox')代表的是以最高权限运行

## 页面加载策略详解：Normal、Eager 和 None 的区别与应用

### Normal（默认策略）

### 定义：Normal 是 Selenium 的默认加载策略。它会等待页面中的所有资源（包括 CSS、图片、JavaScript 等）加载完成后再执行后续操作。
适用场景：适用于需要确保页面完全加载后再进行操作的场景。
特点：
等待页面完全加载，包括所有资源。
适用于需要操作动态加载内容（如 AJAX）的场景。

### Eager

### 定义：Eager 策略会在 DOM（Document Object Model）加载完成后立即执行后续操作，而不会等待页面中的 CSS、图片、JavaScript 等资源加载完成。
适用场景：适用于需要快速加载页面并进行操作的场景。
特点：
等待 DOM 加载完成后立即执行。
提高测试脚本的执行效率。

### None

### 定义：None 策略不会等待页面加载完成，而是立即执行后续操作。它只等待页面的初始 HTML 内容加载完成。
适用场景：适用于需要在页面部分加载时就开始操作的场景。
特点：
不等待页面加载完成。
提高测试脚本的执行效率，但可能需要额外的等待逻辑。

## 测试一个页面的加载速度实际例子

假设需要测试一个页面的加载速度，但页面中包含大量图片和 JavaScript 脚本。如果使用 Normal 策略，测试脚本的执行效率会较低。因此，可以尝试使用 Eager 或 None 策略。

In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# 使用 Eager 策略
options = Options()
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(options=options)
start_time = time.time()
driver.get("https://example.com")
end_time = time.time()
print(f"Page loaded in {end_time - start_time} seconds")
driver.quit()

Page loaded in 0.7138986587524414 seconds


可以看到，相对于normal方式，网页的很多图片甚至还没加载出来，浏览器就已经自动关闭，这在测试的时候可以节省大量的时间

### 使用代理
实际使用中可能有部分网站需要代理才能访问，并且希望设置成只有这部分网站使用代理，可以进行如下配置

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType

proxy = Proxy({
    'proxyType': ProxyType.MANUAL,
    'httpProxy': 'corp-proxy.example.com:8080',
    'sslProxy': 'corp-proxy.example.com:8080',
    'noProxy': 'localhost,127.0.0.1'  # 排除代理的地址
})

options = webdriver.ChromeOptions()
options.proxy = proxy
driver = webdriver.Chrome(options=options)


配置内容类似与虚拟机Ubuntu的手动代理配置，需要将主机上的VPN设置监听特定端口，然后设置端口转发，由于不同VPN甚至不同版本操作都不相同，这里不赘述

implicit	会话隐式等待超时的整数（毫秒）
pageLoad    会话页面加载超时的整数（毫秒）
script      会话脚本超时的整数（毫秒）

后两者的默认值均为30000，而隐式等待默认为0，这就又与刚开始的page_load_strategy有关，implicit  0配合normal，否则容易查找不到动态加载的元素

### acceptInsecureCerts

此功能检查在会话期间导航时 是否使用了过期的 (或) 无效的 TLS Certificate .

如果将功能设置为 false, 则页面浏览遇到任何域证书问题时, 将返回insecure certificate error . 如果设置为 true, 则浏览器将信任无效证书.

默认情况下, 此功能将信任所有自签名证书. 设置后, acceptInsecureCerts 功能将在整个会话中生效.

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

## 将 detach 参数设置为 true 将在进程结束后保持浏览器打开，

这其实类似于ssh远程访问游戏linux服务器一样，通常情况下会话的生命周期同你的ssh连接，一旦你关闭连接，游戏进程就会自动结束，这时候就需要detach来使进程的生命周期独立于ssh窗口，即脱离主线程。
在这个实例中，只要不关闭自动打开的浏览器，selenium就会以后台进程的形式一直运行

In [ ]:
options.add_experimental_option("detach", True)

## 下面是selenium窗口的弹出窗口阻止程序，即你程序打开的浏览器无弹窗

In [ ]:
options.add_experimental_option('excludeSwitches', ['disable-popup-blocking'])

##  Service class

更改日志输出以保存到特定文件

In [ ]:
log_path = r"G:/log"#自定义输出地点
service = webdriver.ChromeService(log_output=log_path)

将日志输出更改为在控制台中显示为 STDOUT

In [ ]:
import subprocess
service = webdriver.ChromeService(log_output=subprocess.STDOUT)

### 共有 6 个可用的日志级别： ALL 、 DEBUG 、 INFO 、 WARNING 、 SEVERE 和 OFF

In [ ]:
import subprocess
service = webdriver.ChromeService(service_args=['--log-level=DEBUG'], log_output=subprocess.STDOUT)